In [ ]:
# 城市名称
# 机场名称
# 航司名称
# 飞机型号
# 飞机注册号

In [1]:
from utils import *
import pandas as pd
from tqdm import tqdm
from Q_templates import Q_templates
import json

In [2]:
connection = connect_to_database("ai_database_dev")
cursor = connection.cursor()

In [3]:
connection1 = connect_to_database("data_center_dev")
cursor1 = connection1.cursor()

In [4]:
connection2 = connect_to_database("data_center_release")
cursor2 = connection2.cursor()

In [6]:
city_names_airport = [_.strip() for _ in open("meta_data/city_name_airport.txt").readlines()]
city_names_supplier = [_.strip() for _ in open("meta_data/city_name_supplier.txt").readlines()]
aircraft_registrations = [_.strip() for _ in open("meta_data/aircraft_registration.txt").readlines()]
aircraft_model_names = [_.strip() for _ in open("meta_data/aircraft_model_name.txt").readlines()]
supplier_names = [_.strip() for _ in open("meta_data/supplier_name.txt").readlines()]
airport_names = [_.strip() for _ in open("meta_data/airport_name.txt").readlines()]

In [ ]:
Q_template = "{city_name}有哪些机场？请给出机场名称和对应的四字码。"
SQL_template = \
'''SELECT DISTINCT airport_name, icao_code FROM airport_fbo_details WHERE city_name = "{}";'''

dataset = []

for city_name in tqdm(city_names_airport):
    Q = Q_template.replace("{city_name}", city_name)
    SQL = SQL_template.format(city_name)
    cursor.execute(SQL)
    A = pd.DataFrame(cursor.fetchall(), columns=["airport_name", "icao_code"]).values.tolist()
    if A:
        A = [f"机场：{_[0]}, 四字码：{_[1]}" for _ in A]
        dataset.append({"Q": Q, "A": A, "SQL": SQL})

In [ ]:
# 一次性查询所有数据，避免每次循环单独查询
SQL = 'SELECT DISTINCT city_name, airport_name, icao_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['city_name', 'airport_name', 'icao_code'])

# 创建一个包含所有问题和答案的列表
dataset = []
for city_name in tqdm(city_names_airport):
    # 过滤数据
    airports = df[df['city_name'] == city_name]['airport_name'].tolist()
    icao_codes = df[df['city_name'] == city_name]['icao_code'].tolist()
    if airports:
        Q = f"{city_name}有哪些机场？请给出机场名称和对应的四字码。"
        SQL_query = f'SELECT DISTINCT airport_name, icao_code FROM airport_fbo_details WHERE city_name = "{city_name}";'
        A = [f"机场名称：{airport}, 四字码：{icao_code}" for airport, icao_code in zip(airports, icao_codes)]
        dataset.append({"Q": Q, "A": A, "SQL": SQL_query})

import json
with open("synthetic_data/1-4.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


In [ ]:
Q_template = "飞机注册号为{aircraft_registration}的飞机是哪一家公司的？"
SQL_template = \
'''SELECT sd.supplier_name
FROM aircraft_details ad
JOIN supplier_details sd ON ad.supplier_id = sd.supplier_id
WHERE ad.registration = "{}";'''

dataset = []

for aircraft_registration in tqdm(aircraft_registrations):
    Q = Q_template.replace("{aircraft_registration}", aircraft_registration)
    SQL = SQL_template.format(aircraft_registration)
    cursor.execute(SQL)
    A = pd.DataFrame(cursor.fetchall())[0].tolist()
    if A:
        dataset.append({"Q": Q, "A": A, "SQL": SQL})

In [ ]:
# 一次性查询所有数据，避免每次循环单独查询
SQL = '''
SELECT ad.registration, sd.supplier_name
FROM aircraft_details ad
JOIN supplier_details sd ON ad.supplier_id = sd.supplier_id;
'''
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['registration', 'supplier_name'])

# 创建一个包含所有问题和答案的列表
dataset = []
for aircraft_registration in tqdm(aircraft_registrations):
    # 过滤数据
    supplier = df[df['registration'] == aircraft_registration]['supplier_name'].tolist()
    if supplier:
        supplier = supplier[0]
        Q = f"飞机注册号为{aircraft_registration}的飞机是哪一家公司的？"
        SQL_query = f'SELECT sd.supplier_name FROM aircraft_details ad JOIN supplier_details sd ON ad.supplier_id = sd.supplier_id WHERE ad.registration = "{aircraft_registration}";'
        dataset.append({"Q": Q, "A": supplier, "SQL": SQL_query})

with open("synthetic_data/1-5.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


In [ ]:
city_names_supplier[190]

In [ ]:
Q_template = "{city_name}的航空公司有哪些？"
SQL_template = \
'''SELECT supplier_name FROM supplier_details WHERE supplier_city_name = "{}";'''

dataset = []

for city_name in tqdm(city_names_supplier):
    Q = Q_template.replace("{city_name}", city_name)
    SQL = SQL_template.format(city_name)
    cursor.execute(SQL)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
    except:
        print(city_name)
        print(A)
        print(SQL)
        continue

with open("synthetic_data/1-6.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{aircraft_model_names}的最大载客量是多少？"
SQL_template = \
'''SELECT max(seat_num) FROM aircraft_details WHERE aircraft_model_name = "{}";'''

dataset = []

for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.replace("{aircraft_model_names}", aircraft_model_name)
    SQL = SQL_template.format(aircraft_model_name)
    cursor.execute(SQL)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
    except:
        print(aircraft_model_name)
        print(A)
        print(SQL)
        continue

with open("synthetic_data/1-7.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{supplier_name}航司的总部位于哪个城市？"
SQL_template = \
'''SELECT supplier_city_name FROM supplier_details WHERE supplier_name = "{}";'''

dataset = []

for supplier_name in tqdm(supplier_names):
    Q = Q_template.replace("{supplier_name}", supplier_name)
    SQL = SQL_template.format(supplier_name)
    cursor.execute(SQL)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
    except:
        print(supplier_name)
        print(A)
        print(SQL)
        continue

with open("synthetic_data/1-8.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 一次性查询所有供应商及其所在城市的数据
SQL = 'SELECT supplier_name, supplier_city_name FROM supplier_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['supplier_name', 'supplier_city_name'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for supplier_name in tqdm(supplier_names):
    try:
        # 在 DataFrame 中过滤出对应的城市
        A = df[df['supplier_name'] == supplier_name]['supplier_city_name'].tolist()[0]
        if A:
            Q = f"{supplier_name}航司的总部位于哪个城市？"
            SQL_query = f'SELECT supplier_city_name FROM supplier_details WHERE supplier_name = "{supplier_name}";'
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing supplier: {supplier_name}, Error: {e}")
        continue

# 将结果保存到文件
output_path = "synthetic_data/1-8.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和SQL查询的模板
Q_template = "{airport_name}机场的三字码是什么？"
SQL_template = '''SELECT iata_code FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场的名称和对应的三字码
SQL = 'SELECT airport_name, iata_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'iata_code'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
    # 过滤出对应机场的三字
        A = df[df['airport_name'] == airport_name]['iata_code'].tolist()[0]
        if A:
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
with open('synthetic_data/1-9.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [11]:
Q_template = "{aircraft_registration}的制造商是谁？"
SQL_template = '''SELECT brand_name FROM aircraft_details WHERE registration = "{}";'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_registration in tqdm(aircraft_registrations):
    Q = Q_template.format(aircraft_registration=aircraft_registration)
    SQL_query = SQL_template.format(aircraft_registration)
    cursor.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-14.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


100%|██████████| 3706/3706 [07:05<00:00,  8.71it/s]


In [ ]:
Q_template = "{aircraft_model_name}的翼展是多少？"
SQL_template = '''SELECT aircraft_width FROM dc_aircraft_model WHERE name = "{}" AND aircraft_width IS NOT NULL;'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor1.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor1.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-20.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{aircraft_model_name}的起飞重量是多少？"
SQL_template = '''SELECT max_takeoff_weight FROM dc_aircraft_model WHERE name = "{}" AND max_takeoff_weight IS NOT NULL;  '''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor1.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor1.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(A)
        print(e)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-21.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [8]:
Q_template = "{aircraft_model_name}属于哪种机型类型（如远程、小飞机）？"
SQL_template = '''SELECT aircraft_type_match.aircraft_type FROM aircraft_type_match INNER JOIN dc_aircraft_model ON aircraft_type_match.`code` = dc_aircraft_model.aircraft_type WHERE name = "{}" LIMIT 1;
'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor2.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor2.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-23.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

100%|██████████| 191/191 [00:08<00:00, 22.58it/s]


In [9]:
Q_template = "{aircraft_model_name}的飞行高度范围是多少？"
SQL_template = '''SELECT certified_ceiling FROM dc_aircraft_model WHERE name = "{}" AND certified_ceiling IS NOT NULL LIMIT 1;'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor2.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor2.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-24.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

 13%|█▎        | 24/191 [00:01<00:07, 23.20it/s]

[15545.0]


 17%|█▋        | 33/191 [00:01<00:06, 23.31it/s]

[13716.0]


 22%|██▏       | 42/191 [00:01<00:06, 23.45it/s]

...

[4400.0]
[10668.0]


 94%|█████████▍| 180/191 [00:07<00:00, 23.27it/s]

[13716.0]


100%|██████████| 191/191 [00:08<00:00, 23.31it/s]

[12497.0]
[12497.0]
[10668.0]


In [17]:
Q_template = "Fleet information of {supplier_name}？"
SQL_template = '''SELECT aircraft_details.aircraft_model_name FROM aircraft_details JOIN supplier_details ON aircraft_details.supplier_id = supplier_details.supplier_id WHERE supplier_details.supplier_name = "{}";'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for supplier_name in tqdm(supplier_names):
    Q = Q_template.format(supplier_name=supplier_name)
    SQL_query = SQL_template.format(supplier_name)
    cursor.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-25.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

 99%|█████████▉| 900/909 [01:45<00:01,  8.66it/s]

['Gulfstream G550']
['Gulfstream G550']


 99%|█████████▉| 902/909 [01:45<00:00,  8.57it/s]

['Gulfstream G550']


 99%|█████████▉| 904/909 [01:45<00:00,  8.60it/s]

...

['Bombardier Global 5000']


100%|█████████▉| 908/909 [01:46<00:00,  8.56it/s]

['Bombardier Global Express XRS']
['Bombardier Global Express XRS']


100%|██████████| 909/909 [01:46<00:00,  8.54it/s]

['Bombardier Global Express XRS']


In [ ]:
# 问题和SQL查询的模板
Q_template = "{airport_name}机场的四字码是什么？"
SQL_template = '''SELECT icao_code FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场的名称和对应的四字码
SQL = 'SELECT airport_name, icao_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'icao_code'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
        A = df[df['airport_name'] == airport_name]['icao_code'].tolist()[0]
        if A:
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
with open('synthetic_data/1-10.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和SQL查询的模板
Q_template = "{airport_name}机场在哪个国家哪个城市？"
SQL_template = '''SELECT country_name, city_name FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场的名称、国家和城市
SQL = 'SELECT airport_name, country_name, city_name FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'country_name', 'city_name'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
        # 查找该机场的国家和城市信息
        result = df[df['airport_name'] == airport_name][['country_name', 'city_name']].values
        if result.size > 0:
            country_name, city_name = result[0]
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            A = f"{city_name}, {country_name}"
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-12.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{airport_name}机场的跑道数量是多少？"
SQL_template = '''SELECT runway_num FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场名称和对应的跑道数量
SQL = 'SELECT airport_name, runway_num FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'runway_num'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
        # 查找该机场的跑道数量
        result = df[df['airport_name'] == airport_name]['runway_num'].tolist()
        if result:
            runway_num = result[0]
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            A = int(runway_num)
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Airport: {airport_name} has no runway number available")
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-11.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")

In [ ]:
SQL = 'SELECT DISTINCT city_name, airport_name, icao_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['city_name', 'airport_name', 'icao_code'])
Q_template = "停场在{airport_name}的所有航司有哪些？"
SQL_template = \
'''SELECT
    DISTINCT supplier_name
FROM
    zts_plane_status
WHERE
    status = 0
    AND supplier_name IS NOT NULL
    AND pure_reg IN (
        SELECT
            LOWER(REPLACE(REPLACE(registration, ' ', ''), '-', ''))
        FROM
            dc_aircraft
        WHERE
            delete_tag = 0
    )
    AND arr_icao = "{}" COLLATE utf8mb4_general_ci;'''

dataset = []

for airport_name in tqdm(airport_names):
    try:
        icao_l = df[df['airport_name'] == airport_name]['icao_code'].tolist()
        if len(icao_l)>0:
            icao_code = icao_l[0]
        else:
            continue
        Q = Q_template.replace("{airport_name}", airport_name)
        SQL = SQL_template.format(icao_code)
        cursor1.execute(SQL)
        if cursor1.fetchone() is not None:
            A = pd.DataFrame(cursor1.fetchall())[0].tolist()
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
            print(len(dataset))
    except:
        # print(airport_name)
        # print(A)
        # print(SQL)
        continue

with open("synthetic_data/1-13.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{supplier_name}航司的官网链接是什么？"
SQL_template = '''SELECT supplier_website FROM supplier_details WHERE supplier_name = "{}";'''

# 一次性查询所有供应商的名称和对应的官网链接
SQL = 'SELECT supplier_name, supplier_website FROM supplier_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['supplier_name', 'supplier_website'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for supplier_name in tqdm(supplier_names):
    try:
        # 查找该供应商的官网链接
        result = df[df['supplier_name'] == supplier_name]['supplier_website'].tolist()
        if result:
            website = result[0]
            Q = Q_template.format(supplier_name=supplier_name)
            SQL_query = SQL_template.format(supplier_name)
            A = website
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Supplier: {supplier_name} has no website")
    except Exception as e:
        print(f"Error processing supplier: {supplier_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-16.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型的最大飞行距离是多少？"
SQL_template = '''SELECT max_range FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的最大飞行距离
SQL = 'SELECT aircraft_model_name, max_range FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'max_range'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的最大飞行距离
        result = df[df['aircraft_model_name'] == aircraft_model_name]['max_range'].tolist()
        if result:
            max_range = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            if max_range:
                A = f"{int(max_range)}公里"
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no max range")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-17.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型的座位数是多少？"
SQL_template = '''SELECT seat_num FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的座位数
SQL = 'SELECT aircraft_model_name, seat_num FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'seat_num'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的座位数
        result = df[df['aircraft_model_name'] == aircraft_model_name]['seat_num'].tolist()
        if result:
            seat_num = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            A = seat_num
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no seat number")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-18.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型的机身长度是多少？"
SQL_template = '''SELECT aircraft_length FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的机身长度
SQL = 'SELECT aircraft_model_name, aircraft_length FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'aircraft_length'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的机身长度
        result = df[df['aircraft_model_name'] == aircraft_model_name]['aircraft_length'].tolist()
        if result:
            aircraft_length = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            if aircraft_length:
                A = f"{aircraft_length} meters"  # 假设机身长度的单位为“meters”
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no aircraft length")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-19.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "注册号为{aircraft_registration}的飞机的制造年份是什么？"
SQL_template = '''SELECT make_year FROM aircraft_details WHERE registration = "{}";'''

# 一次性查询所有飞机注册号和对应的制造年份
SQL = 'SELECT registration, make_year FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['registration', 'make_year'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_registration in tqdm(aircraft_registrations):
    try:
        # 查找该飞机注册号的制造年份
        result = df[df['registration'] == aircraft_registration]['make_year'].tolist()
        if result:
            make_year = result[0]
            Q = Q_template.format(aircraft_registration=aircraft_registration)
            SQL_query = SQL_template.format(aircraft_registration)
            if make_year:
                A = f"{int(make_year)}年"
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft registration: {aircraft_registration} has no make year")
    except Exception as e:
        print(f"Error processing registration: {aircraft_registration}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-22.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型属于哪种机型类型（如远程、小飞机）？"
SQL_template = '''SELECT aircraft_type FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的机型类型
SQL = 'SELECT aircraft_model_name, aircraft_type FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'aircraft_type'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的机型类型
        result = df[df['aircraft_model_name'] == aircraft_model_name]['aircraft_type'].tolist()
        if result:
            aircraft_type = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            A = aircraft_type
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no aircraft type")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-16.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
dataset = {}

for Q_template in Q_templates:
    Q = Q_template.replace("{aircraft_registration}", aircraft_registration)
    SQL = SQL_template.format(aircraft_registration)
    cursor.execute(SQL)
    A = pd.DataFrame(cursor.fetchall())[0].tolist()
    if A:
        dataset.append({"Q": Q, "A": A, "SQL": SQL})